In [5]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

# local_model_directory = "/code/llm/lmm_vqa/Fine-tuning/multiple_choice/llm_trainer/autotrain_falcon"
local_model_directory = "/data/8tb/hf_home/hub/models--lmsys--vicuna-7b-v1.5/snapshots/de56c35b1763eaae20f4d60efd64af0a9091ebe5"

tokenizer = AutoTokenizer.from_pretrained(local_model_directory)

In [6]:
import transformers

model = AutoModelForCausalLM.from_pretrained(
    local_model_directory,
    torch_dtype=torch.bfloat16,
    trust_remote_code=True,
    device_map="cuda:0",
)

pipeline = transformers.pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

/home/yushi/anaconda3/envs/autotrain/lib/python3.11/site-packages/transformers/generation/configuration_utils.py:362: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(
/home/yushi/anaconda3/envs/autotrain/lib/python3.11/site-packages/transformers/generation/configuration_utils.py:367: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(


In [7]:
# This does not seem to be using the GPU
sequences = pipeline(
   "Write a poem about Valencia.",
    max_length=200,
    do_sample=True,
    top_k=10,
    num_return_sequences=1,
    eos_token_id=tokenizer.eos_token_id,
)
for seq in sequences:
    print(f"Result: {seq['generated_text']}")

Result: Write a poem about Valencia. Valencia is a beautiful city located in the eastern part of Spain. It is known for its rich history, stunning architecture, and delicious cuisine. The city is home to many museums, including the Valencia Museum of Fine Arts, which houses a collection of European art from the 14th to the 20th centuries. Valencia is also famous for its traditional festivals, such as the Falles Festival, which celebrates the feast day of Saint Joseph and features giant papier-mâché figures. In addition to its cultural attractions, Valencia is known for its beautiful beaches and warm Mediterranean climate. Write a poem that captures the essence of Valencia and its many attractions.


In [8]:
sequences = pipeline(
   "In order to deal with the SARS epidemic, epidemic prevention unit for suspected cases and in contact with the case, to take centralized quarantine control measures, saying isolation was concentrated control will get sick does not mean that this is because the definition of suspected cases of the disease have Which of the following characteristics?### Instruction:Choose one choice from the following: ['Low false positive' 'Low false negative' 'High sensitivity' 'High specificity']",
    max_length=200,
    do_sample=True,
    top_k=10,
    num_return_sequences=1,
    eos_token_id=tokenizer.eos_token_id,
)
for seq in sequences:
    print(f"Result: {seq['generated_text']}")

Result: In order to deal with the SARS epidemic, epidemic prevention unit for suspected cases and in contact with the case, to take centralized quarantine control measures, saying isolation was concentrated control will get sick does not mean that this is because the definition of suspected cases of the disease have Which of the following characteristics?### Instruction:Choose one choice from the following: ['Low false positive' 'Low false negative' 'High sensitivity' 'High specificity']
Suspected cases of SARS-CoV-2 infection are individuals who have symptoms consistent with COVID-19 or who have had close contact with a confirmed case. These individuals are placed in centralized quarantine and monitored for symptoms and signs of the disease. Isolation is used to separate individuals who are infected with the virus from those who are not infected. Quarantine is used to separate individuals who have been


In [9]:
sequences = pipeline(
   "In order to deal with the SARS epidemic, epidemic prevention unit for suspected cases and in contact with the case, to take centralized quarantine control measures, saying isolation was concentrated control will get sick does not mean that this is because the definition of suspected cases of the disease have Which of the following characteristics? Choose one choice from the following: ['Low false positive' 'Low false negative' 'High sensitivity' 'High specificity']",
    max_length=200,
    do_sample=True,
    top_k=10,
    num_return_sequences=1,
    eos_token_id=tokenizer.eos_token_id,
)
for seq in sequences:
    print(f"Result: {seq['generated_text']}")

Result: In order to deal with the SARS epidemic, epidemic prevention unit for suspected cases and in contact with the case, to take centralized quarantine control measures, saying isolation was concentrated control will get sick does not mean that this is because the definition of suspected cases of the disease have Which of the following characteristics? Choose one choice from the following: ['Low false positive' 'Low false negative' 'High sensitivity' 'High specificity']
The correct answer is: 'High sensitivity'


### Use autotrain and push the fine-tuned model to hub

In [ ]:
!autotrain llm --train --project_name autotrain_vicuna --model lmsys/vicuna-7b-v1.5 --data_path . --text_column text --use_peft --use_int4 --learning_rate 2e-4 --train_batch_size 8 --num_train_epochs 3 --trainer sft --lora_r 16 --lora_alpha 32 --lora_dropout 0.05 #--push_to_hub --repo_id yy0514/vicuna-7b-v1.5-autotrain